In [1]:
import wandb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

api = wandb.Api()


In [ ]:
# %debug
# %pdb on

runs = api.runs("semantic-latents/does")
for run in runs:
    print(f"Run ID: {run.id}, Name: {run.name}")

# run1 = api.run("username/project_name/run_id")
# run2 = api.run("username/project_name/run_id")
# run3 = api.run("username/project_name/run_id")
# run4 = api.run("username/project_name/run_id")


In [ ]:
print(run.summary.keys())
history = run.history()
tmp_df = pd.DataFrame(history)
print(tmp_df.info())
print(tmp_df.head())
# print(df.columns)

In [10]:
# def process_run_data(run_data_list, metric_name='episode/score'):
#     # Ensure all DataFrames have the same length
#     min_length = min(len(df) for df in run_data_list)
#     aligned_data = [df.iloc[:min_length] for df in run_data_list]

#     # Stack all DataFrames and calculate mean and std
#     stacked_data = np.stack([df[metric_name].values for df in aligned_data])
#     mean_data = np.mean(stacked_data, axis=0)
#     std_data = np.std(stacked_data, axis=0)

#     return mean_data, std_data

def calculate_ewm(domain, index, data, window=5):
    weights_matrix = np.exp(-np.abs(domain[:, np.newaxis] - index) / window)
    weighted_sums = np.sum(weights_matrix * data[:, np.newaxis], axis=0)
    normalization_factors = np.sum(weights_matrix, axis=0)
    ema_values = weighted_sums / (normalization_factors + np.finfo(float).eps)
    return ema_values


def process_run_data(
    runs,
    var_value,
    variable='replay',
    index_col='_step',
    metric_name='episode/score',
    ema_window=5,
    lim_runs=None,
    data_start=0,
    data_limit=1e6,
):
    all_data = []

    # Create a new regular index over a linear range
    linear_index = np.linspace(start=data_start, stop=data_limit, num=101, dtype=int)
    main_df = pd.DataFrame(columns=['Steps',metric_name,'run'])

    num_runs = 0
    for run in runs:
        if lim_runs is not None and num_runs > lim_runs:
            print(f'limiting to {lim_runs} runs')
            break
        if all(run.config[var] == var_val for var, var_val in zip(variable,var_value)):
            history = run.history()
            tmp_df = pd.DataFrame(history)
            # tmp_df = tmp_df.add_suffix(str(run.id))
            # metric_ref = metric_name + str(run.id)
            # index_ref = index_col + str(run.id)

            # Filter out NaNs and infs
            tmp_df = tmp_df[~tmp_df[metric_name].isin([np.nan, np.inf, -np.inf])]

            # Calculate the EMA for each point in the linear index
            tmp_df[metric_name] = tmp_df[metric_name].ewm(span=ema_window, adjust=False).mean()
            # ewm_values = calculate_ewm(
            #     domain=linear_index,
            #     index=tmp_df[metric_name].to_numpy(),
            #     data=tmp_df[metric_name].to_numpy(),
            #     window=ema_window,
            # )

            interp_values = np.interp(linear_index,tmp_df[index_col], tmp_df[metric_name])


            run_df = pd.DataFrame({
                'Steps': linear_index,
                metric_name: interp_values,
                'run': run.id  # Add run identifier
            })

            main_df = pd.concat([main_df, run_df], ignore_index=True)


            # Apply EMA smoothing

            num_runs += 1

    print(main_df.head())
    return main_df

    # Combine all runs into a single DataFrame
    # combined_data = pd.concat(all_data, axis=1)
    # combined_data = combined_data.set_index('_step')

    # Calculate mean and std
    # mean_data = combined_data.mean(axis=1)
    # std_data = combined_data.std(axis=1)

    # print(mean_data.head())

    # return mean_data, std_data


In [28]:
# # Prepare data structures to hold our results
# uniform_data = []
# cr_data = []

# # Iterate through runs and collect data
# for run in runs:
#     if run.config['replay'] == 'uniform':
#         uniform_data.append(pd.DataFrame(run.history()))
#     elif run.config['replay'] == 'curious-replay':
#         cr_data.append(pd.DataFrame(run.history()))

# uniform_mean, uniform_std = process_run_data(uniform_data)
# cr_mean, cr_std = process_run_data(cr_data)

# # Get x-axis values (assuming all runs have same number of steps)
# x_values = uniform_data[0]['_step'].iloc[:len(uniform_mean)]


In [ ]:
import itertools

# uniform_mean, uniform_std = process_run_data(
#     runs=runs,
#     replay_type='uniform',
#     index_col='_step',
#     metric_name='episode/score',
#     ema_window=1,
#     lim_runs=1,
# )
# cr_mean, cr_std = process_run_data(
#     runs=runs,
#     replay_type='curious-replay',
#     index_col='_step',
#     metric_name='episode/score',
    # ema_window=1,
#     lim_runs=1,
# )
methods = {'replay':['uniform', 'curious-replay']}
mean_dfs = []
std_dfs = []
methods_data = {}
var_values = methods.values()
variables = tuple(methods.keys())
for combo in itertools.product(*var_values):
    print(combo)
    methods_data[combo] = process_run_data(
        runs=runs,
        var_value=combo,
        variable=variables,
        index_col='_step',
        metric_name='episode/score',
        ema_window=1,
        # lim_runs=1,
    )
    # mean_data, std_data = process_run_data(
    #     runs=runs,
    #     var_value=replay_method,
    #     index_col='_step',
    #     metric_name='episode/score',
    #     ema_window=10,
    #     # lim_runs=1,
    # )
    # mean_dfs.append(mean_data)
    # std_dfs.append(std_data)


In [ ]:

import seaborn as sns

plt.figure(figsize=(12, 6))

# Melt the DataFrame to long format


for i, replay_method in enumerate(methods):
    df = methods_data[replay_method]
    # df_wide = df.pivot(index=df.index, columns='runs', values='Cumulative Reward')
    # df_melted = df.melt(var_name='Columns', value_name='Values')
    # print(df_melted.head())
    # sns.lineplot(x='Columns', y='Values', data=df_melted) #, errorbar='sd', estimator='mean')
    sns.lineplot(df, x='Steps', y='episode/score', label=replay_method, estimator='mean', errorbar='sd')

    # plt.fill_between(mean_dfs[i].index,
    #                  mean_dfs[i] - std_dfs[i],
    #                  mean_dfs[i] + std_dfs[i],
    #                  alpha=0.3)


# # Plot uniform replay data
# plt.plot(uniform_mean.index, uniform_mean.values, label='Uniform Replay (mean)', color='blue')
# plt.fill_between(uniform_mean.index, uniform_mean - uniform_std, uniform_mean + uniform_std,
#                  alpha=0.3, color='blue')

# # Plot CR replay data
# plt.plot(cr_mean.index, cr_mean.values, label='Curious Replay (mean)', color='red')
# plt.fill_between(cr_mean.index, cr_mean - cr_std, cr_mean + cr_std,
#                  alpha=0.3, color='red')

# sns.lineplot(x="step", y="loss", data=metrics)
# sns.scatterplot(x="train_accuracy", y="val_accuracy", data=metrics)
# sns.heatmap(corr_matrix, annot=True, cmap="YlOrRd")



# # Plot uniform replay data
# plt.plot(x_values, uniform_mean, label='Uniform Replay (mean)', color='blue')
# plt.fill_between(x_values, uniform_mean - uniform_std, uniform_mean + uniform_std,
#                  alpha=0.3, color='blue')

# # Plot CR replay data
# plt.plot(x_values, cr_mean, label='CR Replay (mean)', color='red')
# plt.fill_between(x_values, cr_mean - cr_std, cr_mean + cr_std,
#                  alpha=0.3, color='red')

plt.xlabel('Step')
plt.ylabel('Your Metric')
plt.title('Comparison of Replay Methods')
plt.legend()
# plt.grid(True, linestyle='--', alpha=0.7)

# plt.tight_layout()
plt.show()
